# Categorical Analysis of Restaurants in LA Downtown and their Online Delivery System

## 1. Introduction

### 1.1 Background

####  Los Angeles is the most populous city of California, second populous city in the entire United 	states of America, after the New York city. Los Angeles is the cultural, financial and commercial center of southern California. Its known for its ethnic diversity and sprawling metropolis. Due to its ethnic diversity Los Angeles is known for diverse cuisines from all over the world. Though there are tons of restaurants to taste around, an increase in traffic and population in Los 	Angeles suburb has increased demand for online food delivery providers and apps. 

#### By analyzing all restaurants around Los Angeles suburbs and their major delivery providers we can get valuable information about the current competitors in the region as well as those restaurants which currently don't have any delivery option and might need such applications in future. We can also study the major cuisines servicing the area currently.

### 1.2. Problem

#### a) To analyze all restaurant venues and extract their major delivery providers if any, to get information on major competitor and at the same time to focus on those restaurants which don't have any delivery providers and might be potential customers for any new delivery providers in future

#### b) To analyze and categorize different cuisines which might provide valuable input for online delivery providers and to new startups planning to open new restaurants

### 1.3. Interest

#### a)This project will be of high interest to the latest trend of online delivery apps and providers who need in depth information of various restaurants and major competitors in the area.

#### b)It will also be beneficial to startups who want to start any new restaurants in the area to get information regarding which major cuisines are there in the area. It could give valuable input on which cuisines are not present in a particular neighborhood and starting those cuisines in the 	areas might prove highly profitable as Los Angeles is a city of diversity and people tend to look for diverse cuisines.

## 2. Data source and acquisition

#### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np

import json
import requests
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#### Geo spatial data of Los Angeles county was provided by datasets of LA county in this link Geo Spatial datasets of LA. From this link I took the latest Geojson file corresponding to neighborhoods of LA.

In [4]:
url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson'

json_data = requests.get(url).json()['features']
json_data[0] ## first sample of response


{'type': 'Feature',
 'properties': {'kind': 'L.A. County Neighborhood (V6)',
  'external_id': 'acton',
  'name': 'Acton',
  'slug': 'acton-la-county-neighborhood-v6',
  'set': '/1.0/boundary-set/la-county-neighborhoods-v6/',
  'metadata': {'city': '',
   'name': 'Acton',
   'region': 'antelope-valley',
   'county': 'los-angeles',
   'type': 'unincorporated-area',
   'slug': 'acton'},
  'resource_uri': '/1.0/boundary/acton-la-county-neighborhood-v6/'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-118.20703400000012, 34.53902300000013],
     [-118.18941400000017, 34.538559000000106],
     [-118.18950600000015, 34.53496300000011],
     [-118.18516200000018, 34.534773],
     [-118.18516400000016, 34.53124600000013],
     [-118.17601500000016, 34.531354],
     [-118.17618900000016, 34.523803],
     [-118.16702500000017, 34.523512],
     [-118.16294000000012, 34.523716],
     [-118.162988, 34.527586],
     [-118.15426700000017, 34.527789],
     [-118.15402700000011, 34.52732],

#### Out of all the location features, regions of LA county, their corresponding Neighborhoods and their coordinates were extracted.

#### The coordinates given in the JSON file were for every Square Kilometer [km²] of neighborhood. Hence, I have taken the mean Latitude and Longitude for every neighborhood while converting it to a data frame

In [6]:
neighborhood_list=[]
for v in range(len(json_data)):
    
    neighborhood_list.append((
     json_data[v]['properties']['metadata']['region'],#region
     json_data[v]['properties']['name'],#neighborhood
     np.mean(json_data[v]['geometry']['coordinates'][0][0][0][1]),# latitude
     np.mean(json_data[v]['geometry']['coordinates'][0][0][0][0]))) # longitude
  
neighborhood_list[0:10] # first 10 samples

[('antelope-valley', 'Acton', 34.53902300000013, -118.20703400000012),
 ('south-la', 'Adams-Normandie', 34.037396, -118.30800200000012),
 ('santa-monica-mountains',
  'Agoura Hills',
  34.168157000000114,
  -118.77621200000014),
 ('northwest-county', 'Agua Dulce', 34.4881090000001, -118.37822400000013),
 ('san-gabriel-valley', 'Alhambra', 34.10504, -118.12174700000018),
 ('beach-cities',
  'Aliso and Wood Regional Park',
  33.58759300000011,
  -117.75272299999989),
 ('south-county', 'Aliso Viejo', 33.60006400000017, -117.734393),
 ('south-bay', 'Alondra Park', 33.89757200000013, -118.32651300000016),
 ('verdugos', 'Altadena', 34.215508000000106, -118.15135400000013),
 ('north-county', 'Anaheim', 33.8705690000001, -117.74465199999989)]

#### convert to dataframe and check for any missing regions or neighborhoods

In [8]:
LA_neighborhood_data = pd.DataFrame(neighborhood_list)
LA_neighborhood_data.columns=['Region','Neighborhood','Latitude','Longitude']
print("Records with neighborhood data missing:{}".format(LA_neighborhood_data[LA_neighborhood_data['Neighborhood'].isnull()==True]))


print("Records with region data missing:{}".format(LA_neighborhood_data[LA_neighborhood_data['Region'].isnull()==True]))

LA_neighborhood_data.head()

Records with neighborhood data missing:Empty DataFrame
Columns: [Region, Neighborhood, Latitude, Longitude]
Index: []
Records with region data missing:Empty DataFrame
Columns: [Region, Neighborhood, Latitude, Longitude]
Index: []


,Region,Neighborhood,Latitude,Longitude
0,antelope-valley,Acton,34.539023,-118.207034
1,south-la,Adams-Normandie,34.037396,-118.308002
2,santa-monica-mountains,Agoura Hills,34.168157,-118.776212
3,northwest-county,Agua Dulce,34.488109,-118.378224
4,san-gabriel-valley,Alhambra,34.105040,-118.121747


#### count of Different neighborhoods within region

In [10]:
LA_neighborhood_data_grpd=LA_neighborhood_data.groupby('Region',as_index=False).count()
LA_neighborhood_data_grpd

#LA_neighborhood_data['Latitude'].min()



,Region,Neighborhood,Latitude,Longitude
0,angeles-forest,3,3,3
1,antelope-valley,13,13,13
2,beach-cities,12,12,12
3,central-la,26,26,26
4,eastside,4,4,4
5,harbor,13,13,13
6,north-county,21,21,21
7,northeast-la,7,7,7
8,northwest-county,12,12,12
9,pomona-valley,3,3,3


#### Get the geolocater information for los angeles city and use the data along with LA neighborhood data to create a map of LA with various regions and neighborhoods

In [12]:
address = ' los angeles city, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of los angeles City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of los angeles City are 33.9255823, -118.2358324.


In [13]:
# create map of New York using latitude and longitude values
map_LA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, region, neighborhood in zip(LA_neighborhood_data['Latitude'], LA_neighborhood_data['Longitude'], LA_neighborhood_data['Region'], LA_neighborhood_data['Neighborhood']):
    
    #print(lat,lng)
    label = '{}, {}'.format(neighborhood, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  
    
map_LA

#### create a subset of data frame consisting of only central LA region to get details corresponding to Los angeles downtown

In [14]:
central_LA=LA_neighborhood_data[LA_neighborhood_data['Region']=='central-la'].reset_index(drop=True)
central_LA



,Region,Neighborhood,Latitude,Longitude
0,central-la,Arlington Heights,34.052504,-118.316715
1,central-la,Beverly Grove,34.085755,-118.372486
2,central-la,Carthay,34.051684,-118.366636
3,central-la,Chinatown,34.067573,-118.224722
4,central-la,Downtown,34.061247,-118.244623
5,central-la,East Hollywood,34.098356,-118.287392
6,central-la,Echo Park,34.095686,-118.244861
7,central-la,Elysian Park,34.093903,-118.242446
8,central-la,Elysian Valley,34.108424,-118.249729
9,central-la,Fairfax,34.088949,-118.351116


## FOURSQUARE API to get restaurant details for the selected neighborhoods corresponding to Downtown LA

In [15]:
CLIENT_ID = 'QJ3LCZ30QJFWZGOFIDWMN5MTLB0DANIGIWOGT3VE3JXHNGL0' # your Foursquare ID
CLIENT_SECRET = 'FTYTSLO110IBMENPFOCH53BNUHUM53XP4UCTQDVS3YRF4LGR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJ3LCZ30QJFWZGOFIDWMN5MTLB0DANIGIWOGT3VE3JXHNGL0
CLIENT_SECRET:FTYTSLO110IBMENPFOCH53BNUHUM53XP4UCTQDVS3YRF4LGR


### Function to get restaurant venues by looping through coordinates of selected neighborhoods using foursquare API.

### Function also extracts cordinates of each venue,its category and delivery provider information

In [17]:
def get_venue(neighborlist,inputlatlist,inputlonglist):
 
 venue_detail=[]
 for neighbor,inputlat,inputlong in zip(neighborlist,inputlatlist,inputlonglist):
    print(neighbor,inputlat,inputlong)
    
    url_req="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{},&query={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,inputlat,inputlong,'Restaurant')
                                                                                                                           
    results=requests.get(url_req).json()     
    results




    venues=results['response']['venues']
    #print( venues[1]['delivery']['provider']['name'])
    for i in range(len(results['response']['venues'])):
    
        if('delivery'  in venues[i]):
           venue_detail.append((
           neighbor,
           inputlat,
           inputlong,
           venues[i]['id'],
           venues[i]['name'],
           venues[i]['location']['lat'],
           venues[i]['location']['lng'],
           venues[i]['categories'][0]['name'],
           venues[i]['delivery']['provider']['name']
           ))
        else:
           venue_detail.append((
           neighbor,
           inputlat,
           inputlong,
           venues[i]['id'],
           venues[i]['name'],
           venues[i]['location']['lat'],
           venues[i]['location']['lng'],
           venues[i]['categories'][0]['name'],
           'No Delivery option'
           ))
    

 La_venues=pd.DataFrame(venue_detail)
 La_venues.columns=['Neighborhood','Latitude','Longitude','Venueid','venuename','venueLatitude','venueLongitude','venuecategory','deliveryoption']
 La_venues
    
 return (La_venues)

#### The neighborhood data is further filtered to Neighborhoods corresponding to downtown region(as seen from folium  map)

#### Foursquare API is used to get restaurant venues for these neighborhoods

In [18]:
central_LA_filtered=central_LA[central_LA['Neighborhood'].isin(['Downtown','Chinatown','Westlake','Pico-Union'])].reset_index(drop=True)



central_LA_filtered

,Region,Neighborhood,Latitude,Longitude
0,central-la,Chinatown,34.067573,-118.224722
1,central-la,Downtown,34.061247,-118.244623
2,central-la,Pico-Union,34.052497,-118.281446
3,central-la,Westlake,34.053889,-118.259068


In [19]:
central_la_venues = get_venue(neighborlist=central_LA_filtered['Neighborhood'],
                                   inputlatlist=central_LA_filtered['Latitude'],
                                   inputlonglist=central_LA_filtered['Longitude']
                                  )

print(len(central_la_venues))

central_la_venues.head(40)

#central_la_venues_grouped=central_la_venues.groupby('Neighborhood')

#get_venue(neighborlist,inputlatlist,inputlonglist):

Chinatown 34.067573 -118.22472200000018
Downtown 34.061247 -118.2446230000002
Pico-Union 34.05249700000014 -118.28144600000013
Westlake 34.05388900000012 -118.25906800000017
120


,Neighborhood,Latitude,Longitude,Venueid,venuename,venueLatitude,venueLongitude,venuecategory,deliveryoption
0,Chinatown,34.067573,-118.224722,52c77412498ee9acc1e90889,Liwan Restaurant & Hookah Lounge,34.050354,-118.437779,Mediterranean Restaurant,grubhub
1,Chinatown,34.067573,-118.224722,49ebc74af964a5202b671fe3,Yang Chow Restaurant,34.062926,-118.238059,Chinese Restaurant,No Delivery option
2,Chinatown,34.067573,-118.224722,4415b1c5f964a520fd301fe3,Full House Seafood Restaurant,34.066155,-118.237916,Chinese Restaurant,No Delivery option
3,Chinatown,34.067573,-118.224722,4b046029f964a520b15322e3,Mayflower Seafood Restaurant,34.059582,-118.238129,Seafood Restaurant,No Delivery option
4,Chinatown,34.067573,-118.224722,49c2818cf964a520f9551fe3,Boda Restaurant,34.075601,-118.217365,Vietnamese Restaurant,No Delivery option
5,Chinatown,34.067573,-118.224722,4aa8416bf964a5206c5020e3,Traxx Restaurant,34.056232,-118.236183,New American Restaurant,No Delivery option
6,Chinatown,34.067573,-118.224722,4a7e6011f964a520d4f11fe3,Noe Restaurant & Bar,34.052290,-118.250963,French Restaurant,No Delivery option
7,Chinatown,34.067573,-118.224722,43d21a6cf964a5200a2e1fe3,Taix French Restaurant,34.078103,-118.261575,French Restaurant,grubhub
8,Chinatown,34.067573,-118.224722,4b87b6a5f964a5204dc831e3,"24/7 Restaurant at The Standard, Downtown LA",34.049743,-118.256548,American Restaurant,No Delivery option
9,Chinatown,34.067573,-118.224722,4c00d9439cf52d7f636714e7,Hae Jang Chon Korean BBQ Restaurant,34.063888,-118.306075,Korean Restaurant,No Delivery option


### This final dataframe is further processed and clustered . This will be explained in next part of assignment.